In [2]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import PIL
from matplotlib import pyplot as plt
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow import keras
import glob, os

In [3]:
!unzip -qq New_Data.zip

In [4]:
!unzip -qq induction-task-2025.zip

In [5]:
!unzip -qq induction-task.zip

In [6]:
!cp /content/New_Data/AI/* /content/Data/Train/AI/

In [22]:
real= glob.glob("/content/Data/Train/Real/*")
fake= glob.glob("/content/Data/Train/AI/*")
print(len(real))
print(len(fake))

400
694


In [23]:
total= real+ fake
print(len(total))

1094


In [24]:
height, width  = [0],[0]
for i in total:
  img = PIL.Image.open(i)  #PIL.Image.open returns an Image object with attributes like format and size
  height.append(np.array(img).shape[0]) #This img can be coverted into an np array. This is a common step and its done cuz np arrays have many usefull properties
  width.append(np.array(img).shape[1])

print(max(height),min(height),sum(height)/len(height))
print(max(width),min(width),sum(width)/len(width))

1024 0 583.2730593607306
1024 0 546.3324200913243


In [25]:
Train_DR= "/content/Data/Train"
Test_DR= "/content/Test_Images"

In [26]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   zoom_range=0.2,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(Train_DR, batch_size=10,target_size=(224, 224))

Found 1094 images belonging to 2 classes.


In [27]:
train_generator.class_indices

{'AI': 0, 'Real': 1}

In [28]:
print (next(train_generator)[0].shape)
input_shap = next(train_generator)[0].shape[1:]
print (input_shap)

(10, 224, 224, 3)
(224, 224, 3)


In [29]:
model1 = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(input_shap)),            #convolutional layer 1: No. of filters=100, filter size=3x3, activation = relu, input_shap = 224x224x3,
    MaxPooling2D(2,2),                                                          #Maxpooling2D filter size=2x2

    Conv2D(64, (3,3), activation='relu'),                                      #convolutional layer 2: No. of filters=100, filter size=3x3, activation = relu,
    MaxPooling2D(2,2),                                                          #Maxpooling2D filter size=2x2

    Conv2D(128, (3,3), activation='relu'),                                      #convolutional layer 2: No. of filters=100, filter size=3x3, activation = relu,
    MaxPooling2D(2,2),

    Flatten(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(train_generator.num_classes, activation='sigmoid')
])
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', 'precision', 'recall'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,177,346 (42.64 MB)

 Trainable params: 11,177,346 (42.64 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
h2= model1.fit(train_generator,epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


110/110 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - acc: 0.6317 - loss: 0.5981 - precision: 0.6470 - recall: 0.6296
Epoch 2/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - acc: 0.7996 - loss: 0.4631 - precision: 0.8007 - recall: 0.7971
Epoch 3/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - acc: 0.8203 - loss: 0.4330 - precision: 0.8208 - recall: 0.8209
Epoch 4/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - acc: 0.7898 - loss: 0.4464 - precision: 0.7873 - recall: 0.7916
Epoch 5/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - acc: 0.7669 - loss: 0.4633 - precision: 0.7631 - recall: 0.7742
Epoch 6/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - acc: 0.8176 - loss: 0.4359 - precision: 0.8229 - recall: 0.8152
Epoch 7/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - acc: 0.8242 - loss: 0.4165 - precision: 0.8283 - recall: 0.8206
Epoch 8/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - acc: 0.8393 - loss: 0.3908 - precision: 0.8342 - recall: 0.8312
Epoch 9/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - acc:

In [32]:
Test_Data= glob.glob("/content/Test_Images/*")
Test_Data[0]

'/content/Test_Images/image_33.jpg'

In [33]:
img_list= []
order_list=[]
blackAndWhiteImages= []
for i in Test_Data:
  im= np.array(PIL.Image.open(i).resize((224,224)))
  if im.shape!=(224,224,3):
      print(i, im.shape)
      blackAndWhiteImages. append([i, im.shape])

  else:
    img_list.append(im)
    print((i[21:])[:-4])
    order_list.append((i[21:])[:-4])

Test_images= np.array(img_list)

image_33
image_46
image_164
image_169
image_39
image_82
image_197
image_170
image_134
image_194
image_6
image_156
image_99
image_146
image_87
image_85
image_150
image_19
image_157
image_121
image_2
image_26
image_0
image_115
image_14
image_69
image_113
image_49
image_98
image_70
image_158
image_173
image_43
image_118
image_152
image_172
image_21
image_185
image_137
image_8
image_94
image_147
image_7
image_167
image_60
image_20
image_124
image_96
image_143
image_97
image_182
image_133
image_171
image_63
image_177
image_67
image_16
image_101
image_18
image_76
image_136
image_66
image_138
image_38
image_77
image_183
image_187
image_159
image_112
image_111
image_120
image_58
image_105
image_191
image_93
image_35
image_3
image_24
image_175
image_48
image_53
image_106
image_34
image_78
image_74
image_151
image_174
image_184
image_129
image_142
image_36
image_65
image_17
image_5
image_166
image_42
image_110
image_50
image_128
image_107
image_13
image_40
image_163
image_1
image_145
image_117
i

In [34]:
predictions= model1.predict(Test_images)
pred_list=[]
for i in range(0, len(Test_images)):
    if predictions[i][1]== 1:
      print(order_list[i][6:], order_list[i], "Real")
      pred_list.append([int(order_list[i][6:]), order_list[i], "Real"])
    else:
      pred_list.append([int(order_list[i][6:]), order_list[i], "AI"])

pred_list.sort(key=lambda x:x[0])
print(pred_list)

7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 902ms/step
33 image_33 Real
46 image_46 Real
164 image_164 Real
169 image_169 Real
39 image_39 Real
82 image_82 Real
197 image_197 Real
170 image_170 Real
134 image_134 Real
194 image_194 Real
6 image_6 Real
156 image_156 Real
99 image_99 Real
146 image_146 Real
87 image_87 Real
85 image_85 Real
150 image_150 Real
19 image_19 Real
157 image_157 Real
121 image_121 Real
2 image_2 Real
26 image_26 Real
0 image_0 Real
115 image_115 Real
14 image_14 Real
69 image_69 Real
113 image_113 Real
49 image_49 Real
98 image_98 Real
70 image_70 Real
158 image_158 Real
173 image_173 Real
43 image_43 Real
118 image_118 Real
152 image_152 Real
172 image_172 Real
21 image_21 Real
185 image_185 Real
137 image_137 Real
8 image_8 Real
94 image_94 Real
147 image_147 Real
7 image_7 Real
167 image_167 Real
60 image_60 Real
20 image_20 Real
124 image_124 Real
96 image_96 Real
143 image_143 Real
97 image_97 Real
182 image_182 Real
133 image_133 Real
171 image_171 Real
63 image_63 Real

In [35]:
import csv
with open('/content/MyPred2.csv', 'w', newline='') as fh:
  writer = csv.writer(fh)
  for i in pred_list:
    writer.writerow([i[1], i[2]])